# Climate Change Sentiment Analysis

#### Sebastian Vargas, Baxter Romero, Johnny Wilcox, Trajan Pei, Caleb Bettcher


## EDA:

Hydrate Data Set $\newline$
Drop any missing long/lattitude rows $\newline$
Create global subset of tweets (COL: Timestamp, Tweet, Sentiment Score)$\newline$
Create subsets of global subset for individual countries$\newline$
Create and evaluate subsets of different countries

Questions we should try to answer with EDA:

Describe details about the dataset you’ve chosen – $\newline$
How many examples(size after analysis), $\newline$
What statistics do you observe about the examples (Both columns, and data shape)$\newline$
How did you create training/validation/test sets? $\newline$


In [1]:
#imports 

import os
import shutil
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 

import keras
import keras.utils
import keras.layers
import keras.models
import keras.optimizers
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import matplotlib.pyplot as plt

In [22]:
# NOTE
# in order to run this project drop in the .csv data set (called: "The Climate Change Twitter Dataset") 
# download at:
# https://www.kaggle.com/datasets/deffro/the-climate-change-twitter-dataset

import pandas as pd

# Read the CSV file
tweets_df = pd.read_csv("The Climate Change Twitter Dataset.csv")

# Display the resulting DataFrame
tweets_df.head() 


,created_at,id,lng,lat,topic,sentiment,stance,gender,temperature_avg,aggressiveness
0,2006-06-06 16:06:42+00:00,6132,NaN,NaN,Weather Extremes,-0.097180,neutral,female,NaN,aggressive
1,2006-07-23 21:52:30+00:00,13275,-73.949582,40.650104,Weather Extremes,0.575777,neutral,undefined,-1.114768,aggressive
2,2006-08-29 01:52:30+00:00,23160,NaN,NaN,Weather Extremes,0.500479,neutral,male,NaN,aggressive
3,2006-11-07 02:46:52+00:00,57868,NaN,NaN,Weather Extremes,0.032816,neutral,male,NaN,aggressive
4,2006-11-27 14:27:43+00:00,304553,NaN,NaN,Importance of Human Intervantion,-0.090428,neutral,male,NaN,aggressive


In [24]:
# checking length
print(len(tweets_df))

# removing NAN values 
tweets_df = tweets_df.dropna()

# checking length again
print(len(tweets_df))


15789411
5307538


In [26]:
# hydrating tweets 

import tweepy

auth = tweepy.OAuth2BearerHandler("")
api = tweepy.API(auth)